<a href="https://colab.research.google.com/github/batust/presidio-db-json/blob/main/pre-prcoessor-data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install presidio_analyzer
!pip install presidio_anonymizer
!python -m spacy download en_core_web_lg

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.9/104.9 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.7/400.7 MB 4.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [2]:
# Trying out batch processing
from typing import List, Optional, Dict, Union, Iterator, Iterable
import collections
from dataclasses import dataclass
import pprint

import pandas as pd

from presidio_analyzer import AnalyzerEngine, BatchAnalyzerEngine, RecognizerResult, DictAnalyzerResult
from presidio_anonymizer import AnonymizerEngine, BatchAnonymizerEngine
from presidio_anonymizer.entities import EngineResult

In [ ]:
# Authenicating and accessing Llama
from os import environ
from huggingface_hub import login

HUGGING_FACE_TOKEN = environ.get("HUGGING_FACE_TOKEN")
login(token=HUGGING_FACE_TOKEN)

In [ ]:
# Keeps crashing session
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

model_name = "meta-llama/Llama-2-7b-chat-hf"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
llama_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer)

def classify_sensitive_columns(columns):
    prompt = f"Given the column names {columns}, identify which may contain sensitive information (e.g., names, IDs, financial data). Return only a list of sensitive column names."

    response = llama_pipeline(prompt, max_length=100, do_sample=True)

    return response[0]['generated_text']

columns = ["Account", "Project ID", "Program Name", "Employee", "Cost Category",
           "Entity", "COA ID", "COA", "Day", "Rate", "QUANTITY", "Actuals",
           "Posting Category", "Description"]

print(classify_sensitive_columns(columns))

In [1]:
!pip install flair
# !pip install presidio_analyzer
# !pip install presidio_anonymizer
# !python -m spacy download en_core_web_lg

In [2]:
from google.colab import files

uploaded = files.upload()
file_name = list(uploaded.keys())[0].strip().replace(' ', '-')

Saving smaller_companies.csv to smaller_companies (1).csv


In [5]:
import pandas as pd
from flair.data import Sentence
from flair.models import SequenceTagger

EXCLUDED_HEADERS = ["description", "comment", "comments", "query", "notes", "email"]

# ner_tagger = SequenceTagger.load("flair/ner-english")
tagger = SequenceTagger.load('flair/pos-english')
if file_name.endswith('.xlsx'):
  df = pd.read_excel(file_name)
elif file_name.endswith('.csv'):
  df = pd.read_csv(file_name)

df

2025-03-19 11:28:24,203 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, S-ORG, S-MISC, B-PER, E-PER, S-LOC, B-ORG, E-ORG, I-PER, S-PER, B-MISC, I-MISC, E-MISC, I-ORG, B-LOC, E-LOC, I-LOC, <START>, <STOP>
2025-03-19 11:28:25,113 SequenceTagger predicts: Dictionary with 53 tags: <unk>, O, UH, ,, VBD, PRP, VB, PRP$, NN, RB, ., DT, JJ, VBP, VBG, IN, CD, NNS, NNP, WRB, VBZ, WDT, CC, TO, MD, VBN, WP, :, RP, EX, JJR, FW, XX, HYPH, POS, RBR, JJS, PDT, NNPS, RBS, AFX, WP$, -LRB-, -RRB-, ``, '', LS, $, SYM, ADD


,id,name,domain,year founded,industry,size range,locality,country,linkedin url,current employee estimate,total employee estimate
0,5872184,ibm,ibm.com,1911.0,information technology and services,10001+,"new york, new york, united states",united states,linkedin.com/company/ibm,274047,716906
1,4425416,tata consultancy services,tcs.com,1968.0,information technology and services,10001+,"bombay, maharashtra, india",india,linkedin.com/company/tata-consultancy-services,190771,341369
2,21074,accenture,accenture.com,1989.0,information technology and services,10001+,"dublin, dublin, ireland",ireland,linkedin.com/company/accenture,190689,455768
3,2309813,us army,goarmy.com,1800.0,military,10001+,"alexandria, virginia, united states",united states,linkedin.com/company/us-army,162163,445958
4,1558607,ey,ey.com,1989.0,accounting,10001+,"london, greater london, united kingdom",united kingdom,linkedin.com/company/ernstandyoung,158363,428960
...,...,...,...,...,...,...,...,...,...,...,...
29995,1201877,hans anders,hansanders.nl,1982.0,retail,1001 - 5000,"giethoorn, overijssel, netherlands",netherlands,linkedin.com/company/hans-anders,390,959
29996,2828041,real canadian superstore,superstore.ca,NaN,retail,1001 - 5000,"madrid, madrid, spain",spain,linkedin.com/company/real-canadian-superstore,390,1320
29997,835865,"shumaker, loop & kendrick, llp",slk-law.com,1925.0,law practice,501 - 1000,"toledo, ohio, united states",united states,linkedin.com/company/shumaker-loop-&-kendrick-llp,390,688
29998,1611480,business wire,businesswire.com,1961.0,public relations and communications,501 - 1000,"san francisco, california, united states",united states,linkedin.com/company/business-wire,390,998


In [ ]:
# Not batch processing
def classify_cell_as_noun(texts):
    sentences = [Sentence(text) for text in texts]
    tagger.predict(sentences)

    results = []
    noun_count = 0

    for sentence, original_text in zip(sentences, texts):
        all_nouns = any("NN" in token.tag for token in sentence)
        classification = "Noun" if all_nouns else "Not a noun"

        results.append({"Original Text": original_text, "Classification": classification})
        noun_count += all_nouns

    is_noun_column = noun_count > (len(results) // 1.5)
    return results, is_noun_column

sensitive = set()

for column in df.columns:
    values = df[column].astype(str).tolist()
    results, is_noun = classify_cell_as_noun(values)

    if is_noun: sensitive.add(column)

print(sensitive)


In [4]:
# Batch processing
import pandas as pd
import concurrent.futures
from flair.data import Sentence
from flair.models import SequenceTagger

tagger = SequenceTagger.load("flair/pos-english")

EXCLUDED_HEADERS = ["description", "comment", "comments", "query", "notes", "email"]
FILE_NAME = "companies_sorted.csv"
BATCH_SIZE = 10000
FLAIR_BATCH_SIZE = 100
MAX_THREADS = 4

sensitive = set()

def classify_cells_as_nouns(texts):
    """Batch processes text to check if columns are noun-heavy."""
    texts = ['' if pd.isna(text) else text for text in texts]

    results = []
    noun_count = 0

    for i in range(0, len(texts), FLAIR_BATCH_SIZE):
        batch_text = texts[i : i + FLAIR_BATCH_SIZE]
        sentences = [Sentence(text) for text in batch_text]

        tagger.predict(sentences)

        for sentence, original_text in zip(sentences, batch_text):
            all_nouns = any("NN" in token.tag for token in sentence)
            classification = "Noun" if all_nouns else "Not a noun"

            results.append({"Original Text": original_text, "Classification": classification})

            noun_count += all_nouns

    is_noun_column = noun_count > (len(results) * 0.66)

    return is_noun_column

def process_column(col, chunk):
    """Runs noun classification on a single column."""
    values = chunk[col].astype(str).fillna("").tolist()
    return col if classify_cells_as_nouns(values) else None

FILE_NAME = 'smaller_companies.csv'
if FILE_NAME.endswith(".xlsx"):
    df_iterator = pd.read_excel(FILE_NAME, chunksize=BATCH_SIZE)
elif FILE_NAME.endswith(".csv"):
    df_iterator = pd.read_csv(FILE_NAME, chunksize=BATCH_SIZE)

for chunk in df_iterator:
    with concurrent.futures.ThreadPoolExecutor(max_workers=MAX_THREADS) as executor:
        futures = {executor.submit(process_column, col, chunk): col for col in chunk.columns}

        for future in concurrent.futures.as_completed(futures):
            result = future.result()
            if result:
                sensitive.add(result)

print("Sensitive Columns:", sensitive)

2025-03-19 11:51:26,893 SequenceTagger predicts: Dictionary with 53 tags: <unk>, O, UH, ,, VBD, PRP, VB, PRP$, NN, RB, ., DT, JJ, VBP, VBG, IN, CD, NNS, NNP, WRB, VBZ, WDT, CC, TO, MD, VBN, WP, :, RP, EX, JJR, FW, XX, HYPH, POS, RBR, JJS, PDT, NNPS, RBS, AFX, WP$, -LRB-, -RRB-, ``, '', LS, $, SYM, ADD


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/rnn.py:1124: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters(). (Triggered internally at /pytorch/aten/src/ATen/native/cudnn/RNN.cpp:1412.)
  result = _VF.lstm(
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/rnn.py:1124: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters(). (Triggered internally at /pytorch/aten/src/ATen/native/cudnn/RNN.cpp:1412.)
  result = _VF.lstm(
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/rnn.py:1124: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, pos

Sensitive Columns: {'linkedin url', 'name', 'country', 'industry', 'locality'}


In [6]:
masked_df = df_iterator.copy()

for col in masked_df:
  if col in sensitive:
    masked_df[col] = [f'{col}_{i}' for i in range(len(masked_df[col]))]

masked_df

AttributeError: 'TextFileReader' object has no attribute 'copy'

In [8]:
!python --version
!pip freeze | grep numpy
!pip freeze | grep transformers

!pip install --no-cache-dir transformers==4.36.2

Python 3.11.11
numpy==1.26.4
sentence-transformers==3.4.1
transformers==4.48.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.8/126.8 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 163.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 335.2 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.1
    Uninstalling tokenizers-0.21.1:
      Successfully uninstalled tokenizers-0.21.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.48.3
    Uninstalling transformers-4.48.3:
      Successfully uninstalled transformers-4.48.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 3.4.1 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.36.2 which is incompatible.
